In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm
import PyPDF2
import re
from os import listdir
from os.path import isfile, join

## URL von allen Hauptseiten holen

In [ ]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,41):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] #erst ab der Position 2 sind die Daten relevant.
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"VorlageNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [ ]:
pd.DataFrame(alle_seiten)

In [2]:
df_vs=pd.read_csv("KRFrames/df_v_Hauptseiten")

In [3]:
df_vs

,Unnamed: 0,index,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,0,0,3020,/gast/geschaefte/2008,Postulat von Luzian Franzini und Esther Haas b...,Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,1,1,3019,/gast/geschaefte/2007,"Motion von Luzian Franzini, Rita Hofer, Tabea ...",Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
2,2,2,3018,/gast/geschaefte/2006,"Interpellation von Fabio Iten, Laura Dittli un...",Überweisung an Regierungsrat,Volkswirtschaftsdirektion,Interpellation,hängig
3,3,3,3017,/gast/geschaefte/2005,"Postulat von Tabea Zimmermann Gibson, Stéphani...",Überweisung an Regierungsrat,Direktion des Innern,Postulat,hängig
4,4,6,3014,/gast/geschaefte/2002,Interpellation der SVP-Fraktion betreffend mob...,Kenntnisnahme,Sicherheitsdirektion,Interpellation,hängig
...,...,...,...,...,...,...,...,...,...
1215,1215,1946,762,/gast/geschaefte/1331,Motion der erweiterten Justizprüfungskommissio...,Datenmigration,NaN,Motion,abgeschlossen
1216,1216,1952,666,/gast/geschaefte/1325,Motion der Kommission Teilrevision Personalges...,Datenmigration,NaN,Motion,abgeschlossen
1217,1217,1968,304,/gast/geschaefte/1309,Motion von Manuela Weichelt betreffend HIV-Prä...,Datenmigration,NaN,Motion,abgeschlossen
1218,1218,1969,282,/gast/geschaefte/1712,Motion von Christoph Hohler betreffend Radstre...,Frist erstreckt,Baudirektion,Motion,hängig


# hier ein Test von Barnaby, wie man die Namen alleine mit "Split" separieren kann

- Vielleicht zuerst noch mit Pandas nur die Vorstösse raussuchen. Ist ja das, was mich interessiert.
- Rausfinden, wie ich die Namen nun rausziehe und in ein aufklappbares Dataframe stelle


In [4]:

def splitting(elem):
    elem = elem.split('betreffend')[0].strip()
    if 'von' in elem:
        elem = elem.split('von')[1].strip()
        elem = elem.replace(' und', ',')
    elif 'der' in elem:
        elem = elem.split('der')[1].strip()    
    return elem
df_vs["Geschäft"]=df_vs['Geschäft'].apply(splitting) #.apply ist um eine Funktion auf eine ganze Reihe Pandas einzusetze

# Nun rausfinden, wie ich ein DataFrame machen kann, das man aufklappt

In [5]:
df_vs.head()

,Unnamed: 0,index,VorlageNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,0,0,3020,/gast/geschaefte/2008,"Luzian Franzini, Esther Haas",Überweisung an Regierungsrat,Sicherheitsdirektion,Postulat,hängig
1,1,1,3019,/gast/geschaefte/2007,"Luzian Franzini, Rita Hofer, Tabea Zimmermann ...",Überweisung an Regierungsrat,Direktion des Innern,Motion,hängig
2,2,2,3018,/gast/geschaefte/2006,"Fabio Iten, Laura Dittli, Anna Bieri",Überweisung an Regierungsrat,Volkswirtschaftsdirektion,Interpellation,hängig
3,3,3,3017,/gast/geschaefte/2005,"Tabea Zimmermann Gibson, Stéphanie Vuichard, E...",Überweisung an Regierungsrat,Direktion des Innern,Postulat,hängig
4,4,6,3014,/gast/geschaefte/2002,SVP-Fraktion,Kenntnisnahme,Sicherheitsdirektion,Interpellation,hängig


In [6]:
#das funktionerit also nicht
def splitkomma(elem):
    elem=elem.split(", ")
    return elem


In [7]:
df_vs["Geschäft"]=df_vs["Geschäft"].apply(splitkomma) 

In [39]:
# Hier die Lösung, wie ich für jeden Namen ein einzelnes Geschäft erhalte

In [8]:
df_ex=df_vs.explode('Geschäft').reset_index(drop=True)

In [9]:
df_ex.rename(columns={'Geschäft':'Namenganz'}, inplace=True)

In [11]:
df_ex.to_csv("KRFrames/df_ex")

In [2]:
df_ex=pd.read_csv("KRFrames/df_ex")

In [11]:
# hier schaue ich, wie gross das Dataframe ohne die Kommissionen und so ist.
df_ex_cl=df_ex[df_ex["Namenganz"].str.contains("Kommission|kommission")==False]

In [36]:
df_ex_cl=df_ex_cl[["VorlageNR", "Namenganz", ]]

In [41]:
# ICH bereinige von Hand im Excel
df_ex_cl.to_excel("KRFrames/df_ex_cl.xlsx")

In [39]:
df_ex_cl[df_ex_cl["Namenganz"].str.contains("zur")]
#oook, die Alternativen wurden unteschiedliche benamselt. Das muss ich genauer anschauen.
# Auch andere Parteien werden zum Teil doppelt genannt. 

,VorlageNR,Namenganz
779,2362,Manfred Wenger zur Beantwortung der Kleinen An...
1273,1794,zur Förderung der Hausarztmedizin
1286,1781,FDP-Fraktion zur Abschaffung
1515,1542,Alois Gössi zur Ergänzung des Kantonsratsbesch...
1663,1361,Kathrin Kündig zur Schaffung einer Höheren Fac...
1703,1315,Jean-Pierre Prodolliet zur Interpellation


# Bereinigen der Parteinamen
Diese werden in mindestens drei unterschiedlichen Arten genannt

### Ich habe mich fürs Bereinigen im Excel entschieden, da Daten noch übersichtlich sind

In [212]:
# Hier nun das im Excel bereinigte Sheet
df_ex_ber=pd.read_excel("KRFrames/df_ex_cl_geputzt.xlsx")

### Problem erkannt: 
offenbar werden gekoppelte Namen in den Geschäften nur mit den ersten aufgeführt.

In [213]:
df_ex_ber.set_index("Unnamed: 0")


,VorlageNR,Namenganz
Unnamed: 0,,
0,3020,Luzian Franzini
1,3020,Esther Haas
2,3019,Luzian Franzini
3,3019,Rita Hofer
4,3019,Tabea Zimmermann Gibson
...,...,...
1937,836,Käty Hofer Buser
1940,810,Marcel Meyer
1941,801,Hans Abicht


In [214]:
df_ex_ber.shape

(1915, 3)

In [215]:
# Nun das Frame mit den Namen einlesen
df_np=pd.read_excel('KRFrames/Alle_Namen_mit_Allianz_und_ohne_ergänzt.xlsx')

In [216]:
#von der Bearbeitung hat es nun noch einen anderen Index. stört uns nicht - Wir entfernen "unnamed:0"
del df_np["Unnamed: 0"]

In [217]:
df_np.shape #voila

(410, 3)

In [219]:
#Nun die beiden zusammenfügen - an den Namen
df_alles=pd.merge(df_np, df_ex_ber, how="inner",on= "Namenganz", )

In [220]:
df_alles=df_alles.sort_values("Unnamed: 0")

In [221]:
df_alles

,Namenganz,P,Unnamed: 0.1,Unnamed: 0,VorlageNR
1828,Luzian Franzini,ALG,NaN,0,3020
1018,Esther Haas,ALG,NaN,1,3020
1829,Luzian Franzini,ALG,NaN,2,3019
1231,Rita Hofer,ALG,NaN,3,3019
1323,Tabea Zimmermann Gibson,ALG,NaN,4,3019
...,...,...,...,...,...
110,Käty Hofer Buser,SP,NaN,1937,836
14,Marcel Meyer,CVP,NaN,1940,810
23,Hans Abicht,FDP,NaN,1941,801
25,Manuela Weichelt,ALG,NaN,1944,304


In [222]:
df_alles.sort_values("P").tail(41)# So, es sind noch einige wenige, die offenbar fehlerhaft sind
# Da es nur noch einzelne Positionen betrifft, finde ich es so jetzt ok.
                            

,Namenganz,P,Unnamed: 0.1,Unnamed: 0,VorlageNR
1153,Markus Hürlimann,SVP,NaN,475,2624
506,Beat Stocker,SVP,NaN,1566,1496
1199,Michael Riboni,SVP,NaN,476,2624
1293,Ralph Ryser,SVP,NaN,479,2620
472,Karl Nussbaumer,SVP,NaN,480,2620
1380,SVP-Fraktion,SVP,NaN,436,2667
510,Stephan Schleiss,SVP,NaN,1208,1875
1198,Michael Riboni,SVP,NaN,437,2666
1407,SVP-Fraktion,SVP,NaN,1205,1878
1111,Manfred Wenger,SVP,NaN,790,2356


In [224]:
df_alles.set_index("Unnamed: 0")

,Namenganz,P,Unnamed: 0.1,VorlageNR
Unnamed: 0,,,,
0,Luzian Franzini,ALG,NaN,3020
1,Esther Haas,ALG,NaN,3020
2,Luzian Franzini,ALG,NaN,3019
3,Rita Hofer,ALG,NaN,3019
4,Tabea Zimmermann Gibson,ALG,NaN,3019
...,...,...,...,...
1937,Käty Hofer Buser,SP,NaN,836
1940,Marcel Meyer,CVP,NaN,810
1941,Hans Abicht,FDP,NaN,801


In [225]:
df_alles.dtypes

Namenganz        object
P                object
Unnamed: 0.1    float64
Unnamed: 0        int64
VorlageNR         int64
dtype: object

In [226]:
del df_alles["Unnamed: 0.1"]


In [227]:
# nun das df_alles noch abspeichern für den Endcode
df_alles.to_csv("KRFrames/Frame_mit_Namen_und_Parteien")

# IRRWEGE

In [ ]:

nameliste=df_vs["Geschäft"].tolist()

In [ ]:
nameliste

nameliste[1] range(0:1219)

In [ ]:
len(nameliste)

In [ ]:
lestlist=[]
lestlist.append("suppe")
len(lestlist)

In [ ]:
namenliste_n=[]
counter = 0
zaehler = 0

for alleeintraege in df_vs["Geschäft"]: #hier gehe ich jede Liste im Dataframe an.
    print(alleeintraege)
    name = alleeintraege[zaehler] #ich will sie auslesen und die Namen einzeln.

    if zaehler < len(alleeintraege)-1: #da die Listen unterschiedlich lang sind, versuche ich hier dies zu berücksichtigen
        zaehler=zaehler+1 #es scheint leider nicht zu funktionieren. 
    elif zaehler == (alleeintraege)-1: 
        continue
   
          
    
    minidictnamen={"Name":name}
    namenliste_n.append(minidictnamen)
  

In [ ]:
pd.DataFrame(namenliste_n)

In [ ]:
return tuple(namenliste_n)

In [ ]:
#Ich versuche es mal schriftlich bevor ich den Loop schreibe:
liste=[]
namenliste={"sophie", "anna","julia"}
minidicttest={"ART": "Motion", "NUMMER": 23455, "NAMEN":namenliste}
liste.append(minidicttest)

In [ ]:
df_test=pd.DataFrame(liste)

In [ ]:
df_test

In [ ]:
for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value